In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import openfilegui


In [ ]:
file = openfilegui.gui_fname("").decode()


In [ ]:
df = pd.read_csv(file)
df['Start'] = pd.to_datetime(df['Start'])
df['End'] = pd.to_datetime(df['End'])
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]  # удалаяем битые столбцы
df['Total'] = ((df['End']-df['Start']).dt.total_seconds() / 60).astype(int)

activities = df.Activity.unique()

cdf = pd.read_csv('data/colors.txt')
color_dict = dict(cdf.values)

In [ ]:
def get_by_week(year):
    weeks = range(1, 54)
    dfy = pd.DataFrame(0, index=np.arange(len(weeks)),
                    columns=np.hstack(['week', activities]))
    dfy['week'] = weeks
    dfy = dfy.set_index('week')

    for index, row in df.iterrows():
        if (row.Start.year > year):
            continue
        if (row.Start.year < year):
            break

        dfy[row.Activity].iloc[row.Start.week - 1] += row.Total

    dfy = dfy.loc[:, (dfy != 0).any(axis=0)]  # удаляем столбцы, в которых все нули
    dfy.iloc[:, :] = dfy.iloc[:, :].div(60, axis=0) # minutes to hours

    colors = []

    for c in dfy.columns:
        if c in color_dict:
            colors.append(color_dict[c])
        else:
            # colors.append("orange")
            dfy.drop(columns=[c], inplace=True)
            # print(f"Dropped {c} column!")

    # os.makedirs('data', exist_ok=True)
    # dfy.to_csv('data/' + str(year) + '.csv', index=False)

    return dfy, colors

In [ ]:
def get_by_month(year):
    months = range(1, 13)
    dfy = pd.DataFrame(0, index=np.arange(len(months)),
                       columns=np.hstack(['month', activities]))
    dfy['month'] = months
    dfy = dfy.set_index('month')

    for index, row in df.iterrows():
        if (row.Start.year > year):
            continue
        if (row.Start.year < year):
            break

        dfy[row.Activity].iloc[row.Start.month - 1] += row.Total

    # удаляем столбцы, в которых все нули
    dfy = dfy.loc[:, (dfy != 0).any(axis=0)]
    dfy.iloc[:, :] = dfy.iloc[:, :].div(60, axis=0)  # minutes to hours

    colors = []

    for c in dfy.columns:
        if c in color_dict:
            colors.append(color_dict[c])
        else:
            dfy.drop(columns=[c], inplace=True)

    return dfy, colors

In [ ]:
# import altair as alt
# mask = (df['Start'] > '2023-1-1') & (df['End'] <= '2024-1-1')
# df1 = df.loc[mask]
# alt.Chart(df1).mark_bar().encode(
#     x='Start',
#     x2='End',
#     y='Activity',
#     color=alt.Color('Activity', scale=alt.Scale(scheme='dark2'))
# ).properties(
#     width=600,
#     height=300
# )


In [ ]:
first_year = df.Start.min().year
last_year = df.End.max().year

for year in range(first_year, last_year + 1):
    dfy, colors = get_by_week(year)
    dfy.plot(kind='area', stacked=False, figsize=(
        20, 10), color=colors, linewidth=0)
    plt.title(year)
    plt.ylabel('hours')
    plt.xlabel('week')
    plt.show()

In [ ]:
first_year = df.Start.min().year
last_year = df.End.max().year

for year in range(first_year, last_year + 1):
    dfy, colors = get_by_month(year)
    dfy.plot(kind='area', stacked=False, figsize=(
        20, 10), color=colors, linewidth=0)
    plt.title(year)
    plt.ylabel('hours')
    plt.xlabel('month')
    plt.show()